# Understanding the SPARQL Evaluation Metrics

This document explains the four metrics used to evaluate the correctness of a predicted SPARQL query against a ground-truth standard. Each metric evaluates a different aspect of the query's output, progressing from high-level structural correctness to a stringent, content-level comparison.

### 1. Arity Matching F1

* ***Core Question:*** *"Did the system identify the correct number of concepts to return?"*
* **How it Works:** This metric diagnoses the most basic errors by comparing the number of columns (the arity) in the predicted and oracle tables using an F1-score. It is a quick check of the output's fundamental shape, independent of column names, order, or content.
* **Example Scenarios:**
    * **Correct Arity:** A prediction with 2 columns vs. an oracle with 2 columns results in a perfect score of **1.0**.
    * **Fewer Columns:** A prediction with 1 column vs. an oracle with 2 results in a score of **0.67**.
    * **More Columns:** A prediction with 3 columns vs. an oracle with 2 results in a score of **0.8**.

### 2. Entity Set F1

* ***Core Question:*** *"Did the system retrieve the correct set of entities?"*
* **How it Works:** This metric isolates performance on entity linking, independent of relational structure. It first finds the optimal column alignment by testing all possible permutations. Using this best-fit mapping, it then computes an F1-score on the *set of unique values* within each corresponding column.
* **Key Detail:** Because it performs a full permutation search, this metric is insensitive to the original order of columns in the predicted query. If the prediction has fewer columns than the oracle, a complete mapping is impossible, and the score is correctly **0.0**.

### 3. Row-Matching F1

* ***Core Question:*** *"Are the relationships between entities correctly constructed?"*
* **How it Works:** This stricter metric evaluates the structural correctness of the results. It begins by finding the optimal column alignment via permutation, just like the Entity Set F1. After establishing this best-fit mapping, a predicted row is considered a true positive only if an identical row exists in the oracle. This provides a robust measure of correctness that is flexible to variations in column naming and order.
* **Example Scenarios:**
    * A query that retrieves the correct entities but misses a relational constraint (e.g., a `brick:feeds` relationship) will have a low score (e.g., **0.16**), as most of the generated rows will not exist in the more constrained oracle.
    * Because it finds the best alignment first, the score will be the same even if the predicted query has its columns in a different order.

### 4. Exact-Match F1

* ***Core Question:*** *"Did the results match the oracle perfectly?"*
* **How it Works:** Equivalent to `Execution Accuracy` in Text-to-SQL, this metric provides a final, stringent measure of overall correctness. It requires that the column **order** be identical to the oracle. A positional mapping is created (first predicted column to first oracle column, etc.), and a row-wise F1 score is calculated based on this fixed structure. No column permutation is performed.
* **Example Scenarios:**
    * A query with perfect content and the same column order, but different column *names* (`?e`, `?s` vs. `?eqp`, `?sensor`), correctly receives a perfect score of **1.0**.
    * A query with a different column order (`?s`, `?ahu` vs. `?eqp`, `?sensor`) correctly receives a score of **0.0**.
    * A query with a different number of columns also correctly receives a score of **0.0**.

### Summary Table

| Metric                | What it Checks                          | Strictness | Column Alignment Performed? |
| --------------------- | --------------------------------------- | ---------- | --------------------------- |
| **Arity Matching F1** | The number of columns.                  | Low        | No                          |
| **Entity Set F1** | Sets of unique values within columns.   | Medium     | **Yes (Finds best match)** |
| **Row-Matching F1** | Row-for-row content.                    | High       | **Yes (Finds best match)** |
| **Exact-Match F1** | Column order and row-for-row content.   | Very High  | No (Assumes fixed order)    |

# 1. Testing on generic sparql queries

Below, we run unit tests on generic dummy query results for each metric. 

In [4]:
import numpy as np
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from typing import List, Dict, Any

from metrics import (
    get_arity_matching_f1,
    get_entity_and_row_matching_f1,
    get_exact_match_f1
)

# --- Test Runner ---
def run_tests(test_cases):
    """A simple function to run test cases and report results."""
    passed_count = 0
    failed_tests = []
    
    for test in test_cases:
        func = test["func"]
        args = test["args"]
        expected = test["expected"]
        test_name = test["name"]
        
        try:
            actual = func(*args)
            # Use a tolerance for floating point comparisons
            if np.isclose(actual, expected):
                print(f"✅ PASS: {test_name}")
                passed_count += 1
            else:
                failed_tests.append((test_name, expected, actual))
                print(f"❌ FAIL: {test_name} | Expected: {expected:.4f}, Got: {actual:.4f}")
        except Exception as e:
            failed_tests.append((test_name, expected, f"ERROR: {e}"))
            print(f"💥 ERROR: {test_name} | Exception: {e}")

    print("\n" + "="*50)
    print("--- Test Summary ---")
    if not failed_tests:
        print(f"🎉 All {passed_count} tests passed! 🎉")
    else:
        print(f"🔥 {len(failed_tests)} test(s) failed: 🔥")
        for name, exp, act in failed_tests:
            print(f"  - {name} | Expected: {exp}, Got: {act}")
    print("="*50)

# --- Test Data ---
gold_rows_base = [
    {"name": {"value": "Alice"}, "age": {"value": "30"}},
    {"name": {"value": "Bob"}, "age": {"value": "25"}},
    {"name": {"value": "Charlie"}, "age": {"value": "35"}}
]

# --- Test Case Definitions ---
# Helper wrappers to extract metrics from get_entity_and_row_matching_f1
def entity_set_f1(gt, pred):
    return get_entity_and_row_matching_f1(gt, pred)["entity_set_f1"]

def row_matching_f1(gt, pred):
    return get_entity_and_row_matching_f1(gt, pred)["row_matching_f1"]

all_test_cases = [
    # --- Tests for get_arity_matching_f1 ---
    {"name": "Arity F1: Correct SPARQL", "func": get_arity_matching_f1, "args": ["SELECT ?n ?a WHERE {}", gold_rows_base], "expected": 1.0},
    {"name": "Arity F1: Incorrect SPARQL", "func": get_arity_matching_f1, "args": ["SELECT ?n WHERE {}", gold_rows_base], "expected": 2/3},
    {"name": "Arity F1: Correct Table", "func": get_arity_matching_f1, "args": [gold_rows_base, gold_rows_base], "expected": 1.0},
    {"name": "Arity F1: Incorrect Table", "func": get_arity_matching_f1, "args": [[{"name": {"value": "A"}}], gold_rows_base], "expected": 2/3},
    {"name": "Arity F1: Both Empty", "func": get_arity_matching_f1, "args": [[], []], "expected": 1.0},

    # --- Tests for get_entity_set_f1 (No mapping provided) ---
    {"name": "Entity Set F1: Perfect Match", "func": entity_set_f1, "args": [gold_rows_base, gold_rows_base], "expected": 1.0},
    {"name": "Entity Set F1: Perfect Match (Mapped Names)", "func": entity_set_f1, "args": [gold_rows_base, [{"person": r["name"], "years": r["age"]} for r in gold_rows_base]], "expected": 1.0},
    {"name": "Entity Set F1: Superset Pred", "func": entity_set_f1, "args": [gold_rows_base, gold_rows_base + [{"name": {"value": "David"}, "age": {"value": "40"}}]], "expected": 0.857142},
    {"name": "Entity Set F1: Subset Pred", "func": entity_set_f1, "args": [gold_rows_base, gold_rows_base[:2]], "expected": 0.8},         
    {"name": "Entity Set F1: No Match", "func": entity_set_f1, "args": [gold_rows_base, [{"name": {"value": "Z"}, "age": {"value": "99"}}]], "expected": 0.0},    

    # --- Tests for get_row_matching_f1 (No mapping provided) ---
    {"name": "Row Match F1: Perfect Match", "func": row_matching_f1, "args": [gold_rows_base, gold_rows_base], "expected": 1.0},
    {"name": "Row Match F1: Perfect Match (Mapped Names)", "func": row_matching_f1, "args": [gold_rows_base, [{"p": r["name"], "a": r["age"]} for r in gold_rows_base]], "expected": 1.0},
    {"name": "Row Match F1: Shuffled Order (Mapped Names)", "func": row_matching_f1, "args": [gold_rows_base, [{"a": r["age"], "p": r["name"]} for r in reversed(gold_rows_base)]], "expected": 1.0},
    {"name": "Row Match F1: Subset Pred", "func": row_matching_f1, "args": [gold_rows_base, gold_rows_base[:1]], "expected": 0.5},
    {"name": "Row Match F1: Partial Content Mismatch", "func": row_matching_f1, "args": [gold_rows_base, [{"name": {"value": "Alice"}, "age": {"value": "30"}}, {"name": {"value": "Bob"}, "age": {"value": "99"}}]], "expected": 0.4},

    # --- Tests for get_exact_match_f1 ---
    {"name": "Exact Match F1: Perfect Match", "func": get_exact_match_f1, "args": [gold_rows_base, gold_rows_base], "expected": 1.0},
    {"name": "Exact Match F1: Shuffled Order", "func": get_exact_match_f1, "args": [gold_rows_base, [gold_rows_base[1], gold_rows_base[2], gold_rows_base[0]]], "expected": 1.0},
    {"name": "Exact Match F1: Schema Mismatch", "func": get_exact_match_f1, "args": [gold_rows_base, [{"person": {"value": "Alice"}, "years": {"value": "30"}}]], "expected": 0.5},
    {"name": "Exact Match F1: Subset Pred", "func": get_exact_match_f1, "args": [gold_rows_base, gold_rows_base[:2]], "expected": 0.8},
    {"name": "Exact Match F1: Both Empty", "func": get_exact_match_f1, "args": [[], []], "expected": 1.0},
]


run_tests(all_test_cases)


✅ PASS: Arity F1: Correct SPARQL
✅ PASS: Arity F1: Incorrect SPARQL
✅ PASS: Arity F1: Correct Table
✅ PASS: Arity F1: Incorrect Table
✅ PASS: Arity F1: Both Empty
✅ PASS: Entity Set F1: Perfect Match
✅ PASS: Entity Set F1: Perfect Match (Mapped Names)
✅ PASS: Entity Set F1: Superset Pred
✅ PASS: Entity Set F1: Subset Pred
✅ PASS: Entity Set F1: No Match
✅ PASS: Row Match F1: Perfect Match
✅ PASS: Row Match F1: Perfect Match (Mapped Names)
✅ PASS: Row Match F1: Shuffled Order (Mapped Names)
✅ PASS: Row Match F1: Subset Pred
✅ PASS: Row Match F1: Partial Content Mismatch
✅ PASS: Exact Match F1: Perfect Match
✅ PASS: Exact Match F1: Shuffled Order
✅ PASS: Exact Match F1: Schema Mismatch
✅ PASS: Exact Match F1: Subset Pred
✅ PASS: Exact Match F1: Both Empty

--- Test Summary ---
🎉 All 20 tests passed! 🎉


# 2. Testing on example brick queries.

Below, we run tests on different cases that we can infer from these metrics using example brick queries. We are using GraphDB to host our building graph. Change the SPARQL endpoint to your GraphDB endpoint. You can also change the code to run on rdflib by running SPARQL locally. 

In [ ]:

# --- 1. Configuration ---

# Set the target SPARQL endpoint
building_name = "bldg11"
SPARQL_ENDPOINT = f"<GRAPHDB ENDPOINT>{building_name}" # CHANGE THIS TO YOUR GRAPHDB ENDPOINT

# Define the ground truth and predicted SPARQL queries
gold_query = """
    PREFIX brick: <https://brickschema.org/schema/Brick#>
    SELECT DISTINCT ?eqp ?sensor WHERE {
        ?eqp    a                       brick:Air_Handling_Unit .
        ?vav    a                       brick:VAV .
        ?sensor a                       brick:Zone_Air_Temperature_Sensor .
        ?eqp    brick:feeds             ?vav .
        ?vav    brick:hasPoint          ?sensor .
    }
"""

predicted_query_1 = """
    PREFIX brick: <https://brickschema.org/schema/Brick#>
    SELECT DISTINCT ?ahu ?s  WHERE {
        ?ahu    a        brick:Air_Handling_Unit .
        ?s      a        brick:Zone_Air_Temperature_Sensor .
    }
"""

predicted_query_2 = """
    PREFIX brick: <https://brickschema.org/schema/Brick#>
    SELECT DISTINCT ?s ?ahu   WHERE {
        ?ahu    a        brick:Air_Handling_Unit .
        ?s      a        brick:Zone_Air_Temperature_Sensor .
    }
"""
predicted_query_3 = """
    PREFIX brick: <https://brickschema.org/schema/Brick#>
    SELECT DISTINCT ?eqp ?vav ?sensor WHERE {
        ?eqp    a                       brick:Air_Handling_Unit .
        ?vav    a                       brick:VAV .
        ?sensor a                       brick:Zone_Air_Temperature_Sensor .
        ?eqp    brick:feeds             ?vav .
        ?vav    brick:hasPoint          ?sensor .
    }
"""
predicted_query_4 = """
    PREFIX brick: <https://brickschema.org/schema/Brick#>
    SELECT DISTINCT ?s   WHERE {
        ?ahu    a        brick:Air_Handling_Unit .
        ?s      a        brick:Zone_Air_Temperature_Sensor .
    }
"""
predicted_query_5 = """
    PREFIX brick: <https://brickschema.org/schema/Brick#>
    SELECT DISTINCT ?e ?s WHERE {
        ?e    a                       brick:Air_Handling_Unit .
        ?vav    a                       brick:VAV .
        ?s a                       brick:Zone_Air_Temperature_Sensor .
        ?e    brick:feeds             ?vav .
        ?vav    brick:hasPoint          ?s .
    }
"""


# --- 2. Helper Function to Run Queries ---

def run_sparql_query(query: str, endpoint_url: str) -> List[Dict[str, Any]]:
    """
    Executes a SPARQL query against a remote endpoint and returns the results.
    """
    print(f"🔎 Running query against {endpoint_url}...")
    try:
        sparql = SPARQLWrapper(endpoint_url)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        bindings = results.get("results", {}).get("bindings", [])
        print(f"✅ Query successful. Found {len(bindings)} results.")
        return bindings
    except Exception as e:
        print(f"❌ Query failed: {e}")
        return []

# --- 3. Main Execution and Evaluation ---
print("--- 1. Executing SPARQL Queries ---")
gt_results = run_sparql_query(gold_query, SPARQL_ENDPOINT)
gen_results_1 = run_sparql_query(predicted_query_1, SPARQL_ENDPOINT)
gen_results_2 = run_sparql_query(predicted_query_2, SPARQL_ENDPOINT)
gen_results_3 = run_sparql_query(predicted_query_3, SPARQL_ENDPOINT)
gen_results_4 = run_sparql_query(predicted_query_4, SPARQL_ENDPOINT)
gen_results_5 = run_sparql_query(predicted_query_5, SPARQL_ENDPOINT)

print("\n" + "="*50)
print("--- 2. Calculating Evaluation Metrics ---")

# Metric 1: Arity Matching F1 (No alignment needed)
print("\n--- Arity Matching F1 ---")
arity_f1_1 = get_arity_matching_f1(predicted_query_1, gold_query)
print(f"1. Number of columns match! Value should be 1: {arity_f1_1:.4f}")
arity_f1_3 = get_arity_matching_f1(predicted_query_3, gold_query)
print(f"2. Predictions have more columns than GT. Value should be 0.8: {arity_f1_3:.4f}")
arity_f1_4 = get_arity_matching_f1(predicted_query_4, gold_query)
print(f"3. Predictions have less columns than GT. Value should be 0.667: {arity_f1_4:.4f}")

# Metric 2: Entity Set F1 (Performs internal alignment)
print("\n--- Entity Set F1 ---")
result_1 = get_entity_and_row_matching_f1(gt_results, gen_results_1)
entity_set_f1_1 = result_1["entity_set_f1"]
row_matching_f1_1 = result_1["row_matching_f1"]
print(f"1. Column content mostly match, it should have a high value (0.94): {entity_set_f1_1:.4f}")
result_2 = get_entity_and_row_matching_f1(gt_results, gen_results_2)
entity_set_f1_2 = result_2["entity_set_f1"]
row_matching_f1_2 = result_2["row_matching_f1"]
print(f"2. Column content mostly match (but opposite order of columns), it should have a high value (0.94): {entity_set_f1_2:.4f}")
result_3 = get_entity_and_row_matching_f1(gt_results, gen_results_3)
entity_set_f1_3 = result_3["entity_set_f1"]
row_matching_f1_3 = result_3["row_matching_f1"]
print(f"3. Column content should match identically, it should have a perfect score of 1: {entity_set_f1_3:.4f}")
result_4 = get_entity_and_row_matching_f1(gt_results, gen_results_4)
entity_set_f1_4 = result_4["entity_set_f1"]
row_matching_f1_4 = result_4["row_matching_f1"]
print(f"4. There are fewer columns in predictions, it should be zero: {entity_set_f1_4:.4f}")

# Metric 3: Row Matching F1 (Performs internal alignment)
print("\n--- Row Matching F1 ---")

print(f"1. Relational connection is missing. Column order is same. Thus, row_matching should have low but non zero value (0.16): {row_matching_f1_1:.4f}")
print(f"2. Relational connection is missing. Column order is different. Thus, row_matching should have low but non zero value (0.16): {row_matching_f1_2:.4f}")
print(f"3. Row content should match identically, it should have a perfect score of 1: {row_matching_f1_3:.4f}")
print(f"4. There are fewer columns in predictions, it should be zero: {row_matching_f1_4:.4f}")

# Metric 4: Exact Match F1 (No alignment needed)
print("\n--- Exact Match F1 ---")
exact_match_f1_1 = get_exact_match_f1(gt_results, gen_results_1)
exact_match_f1_2 = get_exact_match_f1(gt_results, gen_results_2)
exact_match_f1_3 = get_exact_match_f1(gt_results, gen_results_3)
exact_match_f1_4 = get_exact_match_f1(gt_results, gen_results_4)
exact_match_f1_5 = get_exact_match_f1(gt_results, gen_results_5)
print(f"1. Exact match should be low but non-zero (0.16) since order matches:  {exact_match_f1_1:.4f}")
print(f"1. Exact match should be zero for other test cases since schema is different:  {exact_match_f1_2:.4f}, {exact_match_f1_3:.4f}, {exact_match_f1_4:.4f}")
print(f"2. Exact match should be 1 since schema is different but order of columns are same: {exact_match_f1_5:.4f}")




--- 1. Executing SPARQL Queries ---
🔎 Running query against http://Ozans-MacBook-Pro-10.local:7200/repositories/bldg11...
✅ Query successful. Found 377 results.
🔎 Running query against http://Ozans-MacBook-Pro-10.local:7200/repositories/bldg11...
✅ Query successful. Found 4268 results.
🔎 Running query against http://Ozans-MacBook-Pro-10.local:7200/repositories/bldg11...
✅ Query successful. Found 4268 results.
🔎 Running query against http://Ozans-MacBook-Pro-10.local:7200/repositories/bldg11...
✅ Query successful. Found 377 results.
🔎 Running query against http://Ozans-MacBook-Pro-10.local:7200/repositories/bldg11...
✅ Query successful. Found 388 results.
🔎 Running query against http://Ozans-MacBook-Pro-10.local:7200/repositories/bldg11...
✅ Query successful. Found 377 results.

--- 2. Calculating Evaluation Metrics ---

--- Arity Matching F1 ---
1. Number of columns match! Value should be 1: 1.0000
2. Predictions have more columns than GT. Value should be 0.8: 0.8000
3. Predictions hav